In [3]:
import re 
import numpy as np 
import pandas as pd
from argparse import Namespace
import pickle 
import torch 

import sys
import os

current_path = notebook_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_path, '..','..'))
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)
    
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from trainer import Trainer
from examples.train_and_visu_non_recurrent import get_multi_ds
from utils.metrics import evaluate_metrics
from jupyter_ipynb.NetMob_training_analysis.utils import load_model_args,load_K_fold_datasets,load_trained_model,get_masked_previous_preds_true,SAVE_FOLDER_PATH


In [1]:
import re 
import numpy as np 
import pandas as pd
from argparse import Namespace
import pickle 
import torch 

import sys
import os

current_path = notebook_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_path, '..','..'))
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)
    
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from trainer import Trainer
from examples.train_and_visu_non_recurrent import get_multi_ds
from utils.metrics import evaluate_metrics
from jupyter_ipynb.NetMob_training_analysis.utils import load_model_args,load_K_fold_datasets,load_trained_model,get_masked_previous_preds_true,SAVE_FOLDER_PATH,L_Apps,get_df_results,get_metrics_from_test
from jupyter_ipynb.NetMob_training_analysis.plotting import plot_boxplot_on_metric


## Load NetMob Metrics : 
folder_name = 're_validation_epsilon100'
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100'
model_args = load_model_args(folder_name)
df1 = get_df_results(trial_id,model_args,L_Apps)

folder_name = 're_validation'
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100'
model_args = load_model_args(folder_name)
df2 = get_df_results(trial_id,model_args,L_Apps)

df_netmob = pd.concat([df1,df2])
df_netmob.head()


## Load Subway-in/Subway-out metrics: 
folder_name = 'tmps'
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271'
model_args = load_model_args(folder_name)

df_subway_out = pd.DataFrame(columns = ['mse','mae','mape','mase','fold','trial_num'])
pattern = re.compile(rf"{trial_id}_\d_")
best_model_names = [name for name in model_args['model'].keys() if pattern.search(name)]

for selected_model_name in best_model_names:
    model_metrics = model_args['model'][selected_model_name]['performance']['test_metrics']
    trial_id = selected_model_name.split('_')[-2]
    if trial_id == 'fcomplete':
        trial_id = selected_model_name.split('_')[-3]
    fold_k = selected_model_name[-1]
    if fold_k == 't':
        fold_k = 4

    df_subway_out.loc[len(df_subway_out)] = [model_metrics['mse'],model_metrics['mae'],model_metrics['mape'],model_metrics['mase'],fold_k,trial_id]
df_subway_out['id'] = 'subway-out'
df_subway_out.head()


df = pd.concat([df_netmob,df_subway_out])
df.head()

Training and Hyper-parameter tuning with Ray is not possible


,mse,mae,mape,mase,fold,id,trial_num
0,1365.452881,21.565752,43.349380,NaN,0,Apple_Video,2
1,1329.773315,21.960958,36.967972,NaN,1,Apple_Video,2
2,1264.973877,20.838892,30.041185,NaN,2,Apple_Video,2
3,1233.327148,20.747341,29.113092,NaN,3,Apple_Video,2
4,1296.285156,20.662750,28.951359,NaN,4,Apple_Video,2


In [12]:
df['id_total'] = df.apply(lambda row : f"{row['fold']}_{row['id']}_{row['trial_num']}",axis=1)
df['id_total'].nunique()
df['id_total'].describe()

count                1750
unique               1745
top       1_Apple_Video_2
freq                    2
Name: id_total, dtype: object

In [21]:
df_duplicates = df[df.duplicated(subset=['id_total'], keep=False)]
df_duplicates

,mse,mae,mape,mase,fold,id,trial_num,id_total
0,1365.452881,21.565752,43.349380,NaN,0,Apple_Video,2,0_Apple_Video_2
1,1329.773315,21.960958,36.967972,NaN,1,Apple_Video,2,1_Apple_Video_2
2,1264.973877,20.838892,30.041185,NaN,2,Apple_Video,2,2_Apple_Video_2
3,1233.327148,20.747341,29.113092,NaN,3,Apple_Video,2,3_Apple_Video_2
4,1296.285156,20.662750,28.951359,NaN,4,Apple_Video,2,4_Apple_Video_2
5,1363.020264,21.467972,36.109715,NaN,0,Apple_Video,2,0_Apple_Video_2
6,1486.801270,23.770555,44.594116,NaN,1,Apple_Video,2,1_Apple_Video_2
7,1254.430298,20.744349,28.216379,NaN,2,Apple_Video,2,2_Apple_Video_2
8,1218.060547,20.714348,27.051476,NaN,3,Apple_Video,2,3_Apple_Video_2
9,1563.096191,23.811346,42.889015,NaN,4,Apple_Video,2,4_Apple_Video_2


In [38]:
folder_name = 're_validation'
save_path = f'{SAVE_FOLDER_PATH}/{folder_name}/best_models'
model_args = pickle.load(open(f'{save_path}/model_args.pkl','rb'))   
model_args['model'].keys()

dict_keys(['subway_in_STGCN_MSELoss_2025_01_20_14_27_20569_f0', 'subway_in_STGCN_MSELoss_2025_01_20_14_27_20569_f1', 'subway_in_STGCN_MSELoss_2025_01_20_14_27_20569_f2', 'subway_in_STGCN_MSELoss_2025_01_20_14_27_20569_f3', 'subway_in_STGCN_MSELoss_2025_01_20_14_27_20569_fcomplete_dataset', 'subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_20_05_38_87836NETMOB_POIS_maps_deezer_insta_concat_early_f0', 'subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_20_05_38_87836concat_early_f0', 'subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_20_05_38_87836NETMOB_POIS_maps_deezer_insta_concat_early_f1', 'subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_20_05_38_87836concat_early_f1', 'subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_20_05_38_87836NETMOB_POIS_maps_deezer_insta_concat_early_f2', 'subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_20_05_38_87836concat_early_f2', 'subway_in_su

In [36]:
df1[df1.id_total == '1_Apple_Video_2']
display(df2[df2.id_total == '1_Apple_Video_3'])
display(df2[df2['trial_num'] == '2'])
df2

,mse,mae,mape,mase,fold,id,trial_num,id_total


,mse,mae,mape,mase,fold,id,trial_num,id_total
5,1363.020264,21.467972,36.109715,NaN,0,Apple_Video,2,0_Apple_Video_2
6,1486.801270,23.770555,44.594116,NaN,1,Apple_Video,2,1_Apple_Video_2
7,1254.430298,20.744349,28.216379,NaN,2,Apple_Video,2,2_Apple_Video_2
8,1218.060547,20.714348,27.051476,NaN,3,Apple_Video,2,3_Apple_Video_2
9,1563.096191,23.811346,42.889015,NaN,4,Apple_Video,2,4_Apple_Video_2


,mse,mae,mape,mase,fold,id,trial_num,id_total
0,1349.168945,21.702271,44.978340,NaN,0,Apple_Video,1,0_Apple_Video_1
1,1319.700439,21.785254,36.126171,NaN,1,Apple_Video,1,1_Apple_Video_1
2,1292.140625,20.880156,29.489262,NaN,2,Apple_Video,1,2_Apple_Video_1
3,1237.414062,21.126162,30.085606,NaN,3,Apple_Video,1,3_Apple_Video_1
4,1312.582520,20.626852,30.769630,NaN,4,Apple_Video,1,4_Apple_Video_1
...,...,...,...,...,...,...,...,...
340,1305.429443,21.249725,39.232002,NaN,0,Fortnite,1,0_Fortnite_1
341,1296.882202,21.731592,36.273453,NaN,1,Fortnite,1,1_Fortnite_1
342,1374.309082,21.300262,28.725454,NaN,2,Fortnite,1,2_Fortnite_1
343,1232.037109,20.894806,30.711609,NaN,3,Fortnite,1,3_Fortnite_1


In [23]:
df_duplicates.set_index('id_total').sort_index()

,mse,mae,mape,mase,fold,id,trial_num
id_total,,,,,,,
0_Apple_Video_2,1365.452881,21.565752,43.349380,NaN,0,Apple_Video,2
0_Apple_Video_2,1363.020264,21.467972,36.109715,NaN,0,Apple_Video,2
1_Apple_Video_2,1329.773315,21.960958,36.967972,NaN,1,Apple_Video,2
1_Apple_Video_2,1486.801270,23.770555,44.594116,NaN,1,Apple_Video,2
2_Apple_Video_2,1264.973877,20.838892,30.041185,NaN,2,Apple_Video,2
2_Apple_Video_2,1254.430298,20.744349,28.216379,NaN,2,Apple_Video,2
3_Apple_Video_2,1233.327148,20.747341,29.113092,NaN,3,Apple_Video,2
3_Apple_Video_2,1218.060547,20.714348,27.051476,NaN,3,Apple_Video,2
4_Apple_Video_2,1296.285156,20.662750,28.951359,NaN,4,Apple_Video,2


In [20]:
df2['id_total'] = df2.apply(lambda row : f"{row['fold']}_{row['id']}_{row['trial_num']}",axis=1)
df2['id_total'].describe()

count              345
unique             345
top       4_Fortnite_1
freq                 1
Name: id_total, dtype: object

In [9]:
folder_name = 're_validation_epsilon100'
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100'
model_args = load_model_args(folder_name)

In [18]:
keys = list(model_args['model'].keys())
[k for k in keys if f"{trial_id}_Apple_Video" in k]

['subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100_Apple_Video_2_f0',
 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100_Apple_Video_2_f1',
 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100_Apple_Video_2_f2',
 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100_Apple_Video_2_f3',
 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100_Apple_Video_2_fcomplete_dataset',
 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100_Apple_Video_3_f0',
 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100_Apple_Video_3_f1',
 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100_Apple_Video_3_f2',
 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100_Apple_Video_3_f3',
 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100_Apple_Video_3_fcomplete_dataset',
 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00

In [8]:
df[df['id_total']=='1_Apple_Video_2']

,mse,mae,mape,mase,fold,id,trial_num,id_total
1,1329.773315,21.960958,36.967972,NaN,1,Apple_Video,2,1_Apple_Video_2
6,1486.801270,23.770555,44.594116,NaN,1,Apple_Video,2,1_Apple_Video_2


In [ ]:
csv_with_mase = df.copy()
csv_with_mase['mase'] = np.arange(len(csv_with_mase))
df.apply(lambda row : csv_with_mase[(csv_with_mase['id'] == row['id']) &
                                    (csv_with_mase['fold'] == row['fold']) &
                                    (csv_with_mase['trial_num'] == row['trial_num']) &
                                    ].iloc[0])

In [2]:
if True:
    save_path = f"MAE_distribution_per_app_and_per_fold.html"
    metric_i = 'mae'
    plot_boxplot_on_metric(df,metric_i,save_path)

    save_path = f"MSE_distribution_per_app_and_per_fold.html"
    metric_i = 'mse'
    plot_boxplot_on_metric(df,metric_i,save_path)

Loading BokehJS ...

Loading BokehJS ...

In [ ]:
model_args = load_model_args(folder_name = 'tmps')
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271'
model_id_i = f"{trial_id}_1_f0"
model_args['model'][model_id_i]['performance']

# Plotting tail of distribution error. Analysis of the associated events

### Load trained Model and the associated datasets:

In [ ]:
# Init
folder_name = 'tmps'
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271'
model_args = load_model_args(folder_name)

# Load DataSets
k_th_fold = 4
model_id_i = f"{trial_id}_1_f{k_th_fold}" if k_th_fold<4 else  f"{trial_id}_1_fcomplete_dataset"
K_subway_ds = load_K_fold_datasets(model_args,model_id_i)

# Select the associated dataset:
ds = K_subway_ds[k_th_fold]

# Load trained model
trainer = load_trained_model(model_args,ds,model_id_i,folder_name)

# Evaluate on the selected dataset:
Preds,Y_true,T_labels = trainer.testing(ds.normalizer)

### Plot distribution of error: 

In [ ]:
q = 0.95
abs_error = torch.abs(Preds - Y_true)
#mask = (abs_error> torch.quantile(abs_error, q))
mask = (abs_error> torch.quantile(abs_error, q,axis=0))

masked_previous,masked_Preds,masked_True = get_masked_previous_preds_true(mask,Preds,Y_true)


dic_metric = evaluate_metrics(Preds,Y_true,metrics=['mse','mae','mape','mase'], alpha = None, type_calib = None,dic_metric = {})
print('Metrics all points: ')
display(dic_metric)


dic_metric = evaluate_metrics(masked_Preds,masked_True,metrics=['mse','mae','mape','mase'], alpha = None, type_calib = None,dic_metric = {},previous=masked_previous)
print('Metrics tail q95:')
display(dic_metric)
pd.DataFrame({'Absolute Error': abs_error[mask]}).plot.hist(bins=1000,title = f'Distribution from q{int(q*100)} of Absolute Error of each station')

## Analysis Dates et Stations concernées:

In [ ]:
dates = ds.tensor_limits_keeper.df_verif_test.iloc[:,-1].reset_index(drop=True)
list_hour,occ_stations = [],{}
for station_i in range(mask.size(1)):
    dates_i = dates[pd.Series(mask[:,station_i,0])]
    list_hour+=list(dates_i.dt.hour.values)
    occ_stations[ds.spatial_unit[station_i]] = [len(dates_i)]

pd.DataFrame({'hours':list_hour}).plot.hist(bins=24,density=True)
pd.DataFrame(occ_stations).transpose().rename(columns = {0:'Occurence'}).sort_values('Occurence',ascending=False).plot.bar()

## Calculate Error relative to a statistics:

In [ ]:
q = 0.95
calendar_group = ['weekday','hour','minute']

# Abs Errors: 
abs_errors = pd.DataFrame(abs(Preds-Y_true).squeeze())
abs_errors.index = dates

# Std of Volume: 
#=== replace moments when std = '0' by '1'. Correspond to night when metro is closed.
df_raw_volume = pd.DataFrame(ds.raw_values)
df_raw_volume.index = ds.df_dates.iloc[:,-1]
df_raw_volume[~df_raw_volume.index.isin(ds.invalid_dates)]

std_tab = df_raw_volume.groupby([getattr(df_raw_volume.index,calendar_i) for calendar_i in calendar_group]).std()
std_tab.index.names = calendar_group
std_tab[std_tab == 0] = 1  

nb_occurrence = df_raw_volume.groupby([getattr(df_raw_volume.index,calendar_i) for calendar_i in calendar_group]).count().iloc[:,0]
nb_occurrence.index.names = calendar_group
#display(std_tab)
#display(nb_occurrence)



# Multi_index abs_errors with hour/minutes: 
calendar_group_bis = ['date'] + calendar_group
tuples_h_m = list(zip(*[getattr(abs_errors.index,calendar_i) for calendar_i in calendar_group_bis]))
multi_index = pd.MultiIndex.from_tuples(tuples_h_m,names = calendar_group_bis)
abs_errors.index = multi_index

# Dividing abs_errros by the associated std: 
index_std = abs_errors.index.droplevel("date")
std_aligned = std_tab.loc[index_std]

normalized_errors = abs_errors.values / std_aligned.values
abs_errors_normalized = pd.DataFrame(normalized_errors, index=abs_errors.index, columns=abs_errors.columns)


# Distribution of errors:
mask_df = abs_errors_normalized>abs_errors_normalized.quantile(q)
masked_error = abs_errors_normalized[mask_df]
np_masked_error = masked_error.values.reshape(-1)
np_masked_error = np_masked_error[~np.isnan(np_masked_error)]
pd.DataFrame({f"Absolute Relative Errror to std":np_masked_error}).plot.hist(bins= 1000,title = f"Distribution of q{q} of absolute error relative to std by ({', '.join(calendar_group)}) of each station")

In [ ]:
list_hour=[]
occ_stations = pd.DataFrame()
for c in masked_error.columns:
    list_hour_i = list(masked_error[c].reset_index().dropna().hour.values)
    list_hour+=list_hour_i
    occ_stations[ds.spatial_unit[c]] = [len(list_hour_i)]
pd.DataFrame({'hours':list_hour}).plot.hist(bins=24,density=True,title = f'Histogram of hour associated to q{q}')
pd.DataFrame(occ_stations).transpose().rename(columns = {0:'Occurence'}).sort_values('Occurence',ascending=False).plot.bar()

In [ ]:
mask_bool = torch.Tensor(mask_df.to_numpy()).bool()
masked_previous,masked_Preds,masked_True = get_masked_previous_preds_true(mask_bool,Preds,Y_true)
dic_metric = evaluate_metrics(masked_Preds,masked_True,metrics=['mse','mae','mape','mase'], alpha = None, type_calib = None,dic_metric = {},previous=masked_previous)
print('Metrics tail q95:')
display(dic_metric)